In [11]:
# !pip install -r requirements.txt

In [12]:
# !pip install -U langchain-huggingface

In [13]:
# ✅ Check if Ollama is running
import requests

def is_ollama_running():
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            return True
    except requests.exceptions.ConnectionError:
        pass
    return False

# Example usage
if is_ollama_running():
    print("✅ Ollama is running.")
else:
    print("❌ Ollama is NOT running. Please start it with: `ollama run llama3`")

✅ Ollama is running.


# 📓 Ask My PDFs – Demo Notebook
This notebook shows how to:
- Embed multiple PDFs
- Run retrieval-augmented generation queries
- Use local LLMs via Ollama

In [7]:
from rag_utils import load_and_split_pdfs, embed_documents, query_ollama
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Step 1: Load and embed documents
docs = load_and_split_pdfs("pdfs")
# vectordb = embed_documents(docs, persist_dir="chroma_db")
vectordb = embed_documents(docs)

# Step 2: Ask a question
retriever = vectordb.as_retriever()
query = "What is the main topic of the paper?"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Step 3: Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: There is no paper mentioned in the provided context. The text appears to be a summary of educational background and professional experience.


In [8]:
# Ask a question
query = "Can you summarize their professional background?"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: Based on the provided context, here is a summary of Jie Gao's professional background:

* Currently working as a Senior Data Scientist at Walmart in Hoboken, New Jersey (since September 2021)
* Has over 5 years of experience in customer and marketing analysis, data integrity, modeling, and visualization
* Awarded "Driven of Result" under in 2023 Winter, rated exemplary during the 1st year in Walmart


In [9]:
# Ask a question
query = "What is this candidate's education level"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: Master's degree. The candidate has a Master in Econometrics and Quantitative Economics from New York University, completed from September 2018 to May 2020.


In [10]:
# Ask a question
query = "Does this candidate has MMM experience"
relevant_docs = retriever.invoke(query)
context = "\n".join([doc.page_content for doc in relevant_docs[:3]])

# Generate answer using Ollama
response = query_ollama(context, query)
print("Answer:", response)

Answer: Yes, according to the context, Jie Gao has MMM (Marketing Mix Modeling) experience. The summary mentions:

* Developed an in-house Bayesian Hierarchy MMM solution to measure the incremental media contribution on driving High Value Action customers.
* Utilized MMM to optimize marketing budgets, achieving an average 3% increase in marketing ROI.

This indicates that Jie Gao has hands-on experience with MMM and its applications in optimizing marketing budgets.
